In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
import requests

In [2]:
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
request_soup = requests.get('https://pubs.acs.org/', headers=headers)
soup = BeautifulSoup(request_soup.text, 'html.parser')

In [3]:
journals_soup = soup.findAll("div", {"class": "quick-search"})

In [4]:
journals = soup.findAll("option")

In [5]:
#def get_link(journal_code):
#    array = []
#    for i in journal_code:
#        link = "https://pubs.acs.org/journal/{}".format(i)
#        array.append(link)
#    return (array)

In [6]:
#links = get_link(journal_code)
#links_temp = get_link(journal_code_temp)
#links_temp

## Journals

In [7]:
#headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
#req_impact = requests.get('https://pubs.acs.org/journal/achre4', headers=headers)
#soup_impact = BeautifulSoup(req_impact.text, 'html.parser')

In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
#from fake_useragent import UserAgent
import time

In [31]:
journal_name = []
journal_code = []
journal_code_temp = []
current_vol = []
current_issue = []
current_date = []
journal_impact = []
journal_citation = []

In [92]:
journal_name = [s.text.strip() for s in journals]
journal_name = journal_name[1:]
    
journal_code = [s['value'] for s in journals]
journal_code = journal_code[1:]
journal_code

['achre4',
 'aabmcb',
 'aaembp',
 'aaemcq',
 'aamick',
 'aanmf6',
 'aapmcd',
 'abseba',
 'accacs',
 'acscii',
 'acbcct',
 'achsc5',
 'acncdm',
 'acsccc',
 'aesccq',
 'aelccp',
 'aeecco',
 'aewcaa',
 'aidcbc',
 'amlccd',
 'amlcef',
 'amclct',
 'ancac3',
 'acsodf',
 'aptsfn',
 'apchd5',
 'ascefj',
 'ascecg',
 'asbcd6',
 'ancham',
 'bichaw',
 'bcches',
 'bomaf6',
 'bipret',
 'cgeabj',
 'cenear',
 'crtoec',
 'chreay',
 'cmatex',
 'cgdefu',
 'enfuem',
 'esthag',
 'estlcu',
 'iepra6.2',
 'iechad',
 'iecac0',
 'iecjc0',
 'iecfa7',
 'iepdaw',
 'iepra6',
 'iecred',
 'iecnav',
 'inocaj',
 'jacsat',
 'jafcau',
 'jceaax',
 'jci001',
 'jceda8',
 'jcics1',
 'jcisd8',
 'jctcce',
 'jcchff',
 'iechad.1',
 'jmcmar.1',
 'jmcmar',
 'jnprdf',
 'joceah',
 'jpchax',
 'jpchax.2',
 'jpcafh',
 'jpcbfk',
 'jpccck',
 'jpclcd',
 'jprobs',
 'jamsef',
 'langd5',
 'mamobx',
 'mpohbp',
 'nalefd',
 'neaca9',
 'orlef7',
 'oprdfk',
 'orgnd7',
 'pnas',
 'iepra6.1',
 'jpchax.1']

In [207]:
from selenium.common.exceptions import NoSuchElementException
def acs_scraper(journals, browser):   
    current_vol = []
    current_issue = []
    current_date = []
    journal_impact = []
    journal_citation = []
    time.sleep(10)
    
    try:
        impact = browser.find_element_by_class_name('jhHeader_impact')
        journal_impact.append(impact.text)
    except NoSuchElementException:
        impact = 'NA'
        
    try:
        citations = browser.find_element_by_class_name('jhHeader_citations')
        journal_citation.append(citations.text)
    except NoSuchElementException:
        citations = 'NA'
        
        
    issue = browser.find_elements_by_xpath('.//div[@class = "jhHeader_meta"]')
    
    for i in issue:
        try:
            current_vol.append(i.find_element_by_xpath('.//span[1]').text)
        except NoSuchElementException:
            current_vol = 'NA'
        #current_vol.append(i.find_element_by_xpath('.//span[1]').text)
        
        try:
            current_issue.append(i.find_element_by_xpath('.//span[2]').text)
        except NoSuchElementException:
            current_issue = 'NA'
        #current_issue.append(i.find_element_by_xpath('.//span[2]').text)
        
        try:
            current_date.append(i.find_element_by_xpath('.//p').text)
        except NoSuchElementException:
            current_date = 'NA'
        #current_date.append(i.find_element_by_xpath('.//p').text)
    
    #journal_impact.append(impact.text)
    #journal_citation.append(citations.text)
    
    d = dict({"impact":journal_impact,"citation" : journal_citation, "current volume" : current_vol, "current issue" : current_issue, "current date" : current_date})
    
    return d 

In [208]:
from collections import defaultdict

journals = []

options = Options()   
browser = webdriver.Firefox()

for i in journal_code:
    url = 'https://pubs.acs.org/journal/'+str(i)
    browser.get(url)
    print("Processing:", i)
    acs_scraper(journals, browser)
    journals.append(acs_scraper(journals, browser))    
    
df_scraper = pd.DataFrame(journals)
df_scraper.to_csv("acs_journals.csv")
#dict_scraper.loc[-1] = dict_scraper  # adding a row
#dict_scraper.index = dict_scraper.index + 1  # shifting index
#dict_scraper = dict_scraper.sort_index()
    #dict_merged = defaultdict(list)
    #print(dict_merged)
    #for k, v in dict_scraper.items():
    #    dict_scraper[k].append(v)
#df_scraper = pd.DataFrame(dict_scraper)
#df_scraper.to_csv("acs_journals.csv")

#d = dict({"impact":journal_impact,"citation" : journal_citation, "current volume" : current_vol, "current issue" : current_issue, "current date" : current_date})
#pd.DataFrame(d)
#dict_scraper.to_csv("acs_journals.csv")
    #print(pd.DataFrame.append(df_scraper, ignore_index=True))
    #df_scraper = pd.DataFrame(dict_scraper)
    #df_scraper.to_csv("acs_journals.csv") 

Processing: achre4
Processing: aabmcb
Processing: aaembp
Processing: aaemcq
Processing: aamick
Processing: aanmf6
Processing: aapmcd
Processing: abseba
Processing: accacs
Processing: acscii
Processing: acbcct
Processing: achsc5
Processing: acncdm
Processing: acsccc
Processing: aesccq
Processing: aelccp
Processing: aeecco
Processing: aewcaa
Processing: aidcbc
Processing: amlccd
Processing: amlcef
Processing: amclct
Processing: ancac3
Processing: acsodf
Processing: aptsfn
Processing: apchd5
Processing: ascefj
Processing: ascecg
Processing: asbcd6
Processing: ancham
Processing: bichaw
Processing: bcches
Processing: bomaf6
Processing: bipret
Processing: cgeabj
Processing: cenear
Processing: crtoec
Processing: chreay
Processing: cmatex
Processing: cgdefu
Processing: enfuem
Processing: esthag
Processing: estlcu
Processing: iepra6.2
Processing: iechad
Processing: iecac0
Processing: iecjc0
Processing: iecfa7
Processing: iepdaw
Processing: iepra6
Processing: iecred
Processing: iecnav
Processing

In [206]:
from collections import defaultdict

journals = []

options = Options()   
browser = webdriver.Firefox()

for i in journal_code_temp:
    url = 'https://pubs.acs.org/journal/bipret'
    browser.get(url)
    print("Processing:", i)
    print (acs_scraper(journals, browser))

Processing: achre4
{'impact': [], 'citation': [], 'current volume': [], 'current issue': [], 'current date': []}
Processing: aabmcb
{'impact': [], 'citation': [], 'current volume': [], 'current issue': [], 'current date': []}


In [209]:
df_scraper['name'] = journal_name

In [210]:
df_scraper

,impact,citation,current volume,current issue,current date,name
0,[Impact Factor 2019: 20.832],"[Citations 2019: 72,738]",[Volume 53],[Issue 7],"[July 21, 2020]",Accounts of Chemical Research
1,[],[],[Volume 3],[Issue 7],"[July 20, 2020]",ACS Applied Bio Materials
2,[],[],[Volume 2],[Issue 7],"[July 28, 2020]",ACS Applied Electronic Materials
3,[Impact Factor 2019: 4.473],"[Citations 2019: 4,671]",[Volume 3],[Issue 7],"[July 27, 2020]",ACS Applied Energy Materials
4,[Impact Factor 2019: 8.758],"[Citations 2019: 214,885]",[Volume 12],[Issue 30],"[July 29, 2020]",ACS Applied Materials & Interfaces
...,...,...,...,...,...,...
81,[Impact Factor 2019: 3.023],"[Citations 2019: 8,196]",[Volume 24],[Issue 7],"[July 17, 2020]",Organic Process Research & Development
82,[Impact Factor 2019: 3.804],"[Citations 2019: 35,680]",[Volume 39],[Issue 14],"[July 27, 2020]",Organometallics
83,[],[],[],[],[],PNAS
84,[],[],[],[],[],Product R&D


In [211]:
df_scraper['code'] = journal_code

In [213]:
df_scraper.to_csv("acs_journals_name.csv")